In [23]:
!pip install opencv-python

  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/38/d2/3e8c13ffc37ca5ebc6f382b242b44acb43eb489042e1728407ac3904e72f/opencv_python-4.8.1.78-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/38.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/38.1 MB 2.0 MB/s eta 0:00:20
   --- ------------------------------------ 3.2/38.1 MB 41.0 MB/s eta 0:00:01
   -------- ------------------------------- 8.4/38.1 MB 67.5 MB/s eta 0:00:01
   -------------- ------------------------- 13.5/38.1 MB 108.8 MB/s eta 0:00:01
   ------------------- -------------------- 18.6/38.1 MB 108.8 MB/s eta 0:00:01
   ---------------------- ----------------- 21.2/38.1 MB 93.9 MB/s eta 0:00:01
   ------------------------ --------------- 23.1/38.1 MB 81.8 MB/s eta 0:00:01
   ------------------------- -------------- 24.2/38.1 MB 59.5 MB/s eta 0:00:01
   ------------------------------ --------- 28.8/38.1 MB 59.5 MB/s eta 0:00:01


In [26]:
import torch
from torch import nn
import torch.optim as optim
from torch.autograd import Variable
from torchvision import models
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from torchvision import transforms
from PIL import Image
import numpy as np
from torchvision import datasets
#from google.colab import drive
import os
from torch.utils.data import Dataset, DataLoader, TensorDataset
import glob
import json
import random
from IPython.display import clear_output
from torch.optim.lr_scheduler import StepLR
#!drive unmount
#!drive mount '/content/drive', force_remount=True
#drive.mount('/content/drive')

In [2]:
def one_hot_group(data,category):
    label_dict = {data: i for i, data in enumerate(category)}
    #print(label_dict)
    data_indices = [label_dict[str(data)]]
    #print(label_dict)
    one_hot = torch.nn.functional.one_hot(torch.tensor(data_indices), len(category))
    #print(one_hot)
    return one_hot
    
def combine_one_hot_labels(*vectors):
    return torch.cat(vectors, dim=0)

def add_extension(img_path):
    if not img_path.endswith(".jpg"):
        img_path += ".jpg"
    return img_path

def one_hot_encode_label(folder_path,file_name,label_name):
    # 라벨 파일 읽기
    file_path = folder_path+"/"+file_name
    with open(file_path, 'r') as file:
        label_data = json.load(file)

    if label_name == "image":
        img_path = label_data["imgName"]
        img_path_with_extension = add_extension(img_path)
        return img_path_with_extension
    
    era = str(label_data["era"])
    era_category = ['1950','1960','1970','1980','1990','2000','2010','2019']
    
    style = label_data["style"]
    style_category = ["ivy","feminine","classic","mods","minimal","popart","space","hippie","disco","military","punk","bold","powersuit","bodyconscious","hiphop","kitsch","lingerie","grunge","metrosexual","cityglam","oriental","ecology","sportivecasual","athleisure","lounge","normcore","genderless"]
    
    gender = label_data["gender"]
    gender_category = ['M','W']
    
    survey_data = label_data["survey"]
    q1 = survey_data["Q1"]
    q1_category = ["1","2","3","4"]
    q2 = survey_data["Q2"] # 데이터가 여러개임
    q2_category = ["1","2","3"]
    q3 = survey_data["Q3"] # 데이터가 여러개임
    q3_category = ["1","2","3","4","5","6","7","8"]
    q411 = survey_data["Q411"]
    q411_category = ["1","2","3"]
    q412 = survey_data["Q412"]
    q412_category = ["1","2"]
    q413 = survey_data["Q413"]
    q413_category = ["1","2"]
    q414 = survey_data["Q414"]
    q414_category = ["1","2"]
    q42xx_category = ["0","1"]
    q5 = survey_data["Q5"]
    q5_category = ["1","2"]

    one_hot_result = ""

    #'''
    if label_name == "era":
        one_hot_result = one_hot_group(era, era_category)
    elif label_name == "style":
        one_hot_result = one_hot_group(style, style_category)
    elif label_name == "gender":
        one_hot_result = one_hot_group(gender, gender_category)
    elif label_name == "q1":
        one_hot_result = one_hot_group(q1,q1_category)
    elif label_name == "q2":
        one_hot_result = one_hot_group(q2,q2_category)
    elif label_name == "q3":
        one_hot_result = one_hot_group(q3,q3_category)
    elif label_name == "q411":
        one_hot_result = one_hot_group(q411,q411_category)
    elif label_name == "q412":
        one_hot_result = one_hot_group(q412,q412_category)
    elif label_name == "q413":
        one_hot_result = one_hot_group(q413,q413_category)
    elif label_name == "q414":
        one_hot_result = one_hot_group(q414,q414_category)
    #'''
    for i in range(0, 16):
        key = f"Q{4201+i}"
        if label_name == f"q{4201+i}":
            one_hot_result = one_hot_group(survey_data[key],q42xx_category)
            break
    '''
    combined_label = combine_one_hot_vectors(
    era_one_hot, style_one_hot, gender_one_hot,
    q1_one_hot, 
        #q2_one_hot, q3_one_hot,
    q411_one_hot, q412_one_hot, q413_one_hot, q414_one_hot, *q42xx_one_hot
    )
    print(combined_label)
    print(combined_label.shape)
    '''
    
    return one_hot_result

def label_one_hot(image_path,label_path,label_name):
    label_list = os.listdir(label_path)
    temp = []
    for file_name in label_list:
        image_name = one_hot_encode_label(label_path,file_name,"image")
        label = []
        for labels in label_name:
            label.extend(one_hot_encode_label(label_path,file_name,labels))
        image = image_path +'/'+image_name
        data = (image,label)
        temp.append(data)

    return temp

In [3]:
def get_subfolder_names(folder_path):
    subfolders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
    return subfolders

#'''
folder_path_training_image = 'data/Training/data'
folder_path_validation_image = 'data/Validation/data'
folder_path_training_label = 'data/Training/label'
folder_path_validation_label = 'data/Validation/label'
subfolder_names_training_image = get_subfolder_names(folder_path_training_image)
subfolder_names_validation_image = get_subfolder_names(folder_path_validation_image)
subfolder_names_training_label = get_subfolder_names(folder_path_training_label)
subfolder_names_validation_label = get_subfolder_names(folder_path_validation_label)
#'''
'''
print("Training image 폴더 안에 있는 모든 서브폴더명:", subfolder_names_training_image)
print("Training label 폴더 안에 있는 모든 서브폴더명:", subfolder_names_training_label)
print("Vaildation image 폴더 안에 있는 모든 서브폴더명:", subfolder_names_validation_image)
print("Vaildation label 폴더 안에 있는 모든 서브폴더명:", subfolder_names_validation_label)
#'''

'\nprint("Training image 폴더 안에 있는 모든 서브폴더명:", subfolder_names_training_image)\nprint("Training label 폴더 안에 있는 모든 서브폴더명:", subfolder_names_training_label)\nprint("Vaildation image 폴더 안에 있는 모든 서브폴더명:", subfolder_names_validation_image)\nprint("Vaildation label 폴더 안에 있는 모든 서브폴더명:", subfolder_names_validation_label)\n#'

In [4]:
label_list = ["era","style","gender","q1","q2","q3","q411","q412","q413","q414","q4201","q4202","q4203","q4204","q4205","q4206","q4207","q4208","q4209","q4210","q4211","q4212","q4213","q4214","q4215","q4216","q5"]
train_data = []
validation_data =[]
#label_list = ["style","gender","q1"]
#'''
for image_path, label_path in zip(subfolder_names_training_image,subfolder_names_training_label):
    train_data.extend(label_one_hot(folder_path_training_image + '/' + image_path , folder_path_training_label +'/'+ label_path,label_list))
    print(f"Training Data '{image_path}' is OK")
    #print(folder_path_training_image + '/' + image_path , folder_path_training_label +'/'+ label_path)
#'''
#'''
for image_path, label_path in zip(subfolder_names_validation_image,subfolder_names_validation_label):
    validation_data.extend(label_one_hot(folder_path_validation_image + '/' + image_path , folder_path_validation_label +'/'+ label_path,label_list))
    print(f"Validation Data '{image_path}' is OK")
    #print(folder_path_vaildation+'/'+ label_path)
#'''

Training Data 'TS_man_1950' is OK
Training Data 'TS_man_1960' is OK
Training Data 'TS_man_1970' is OK
Training Data 'TS_man_1980' is OK
Training Data 'TS_man_1990' is OK
Training Data 'TS_man_2000' is OK
Training Data 'TS_man_2010' is OK
Training Data 'TS_man_2019' is OK
Training Data 'TS_woman_1950' is OK
Training Data 'TS_woman_1960' is OK
Training Data 'TS_woman_1970' is OK
Training Data 'TS_woman_1980' is OK
Training Data 'TS_woman_1990' is OK
Training Data 'TS_woman_2000' is OK
Training Data 'TS_woman_2010' is OK
Training Data 'TS_woman_2019' is OK
Validation Data 'VS_man_1950' is OK
Validation Data 'VS_man_1960' is OK
Validation Data 'VS_man_1970' is OK
Validation Data 'VS_man_1980' is OK
Validation Data 'VS_man_1990' is OK
Validation Data 'VS_man_2000' is OK
Validation Data 'VS_man_2010' is OK
Validation Data 'VS_man_2019' is OK
Validation Data 'VS_woman_1950' is OK
Validation Data 'VS_woman_1960' is OK
Validation Data 'VS_woman_1970' is OK
Validation Data 'VS_woman_1980' is OK


In [5]:
print(train_data[0])

('data/Training/data/TS_man_1950/T_00001_50_ivy_M.jpg', [tensor([1, 0, 0, 0, 0, 0, 0, 0]), tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0]), tensor([1, 0]), tensor([0, 0, 1, 0]), tensor([1, 0, 0]), tensor([0, 0, 0, 0, 1, 0, 0, 0]), tensor([0, 1, 0]), tensor([0, 1]), tensor([0, 1]), tensor([0, 1]), tensor([1, 0]), tensor([1, 0]), tensor([1, 0]), tensor([1, 0]), tensor([0, 1]), tensor([1, 0]), tensor([1, 0]), tensor([0, 1]), tensor([1, 0]), tensor([1, 0]), tensor([0, 1]), tensor([1, 0]), tensor([1, 0]), tensor([1, 0]), tensor([1, 0]), tensor([1, 0])])


In [82]:
# path error check
for index, (image_path, _) in enumerate(train_data):
    if not os.path.exists(image_path):
        print(f"Error at index {index}: File not found - {image_path}")

In [6]:
#2개
class facedataset(Dataset):
    def __init__(self, data, train=True, test=False, transform=None):
        self.transform = transform

        if test:
            self.images = [img for img, labels in data]
            self.labels = [labels for img, labels in data]
            return

        num_samples = len(data)
        num_test_samples = int(0.2 * num_samples)
        num_train_samples = num_samples - num_test_samples

        train_files, test_files = torch.utils.data.random_split(data, [num_train_samples, num_test_samples])

        if train:
            self.images = [img for img, labels in train_files]
            self.labels = [labels for img, labels in train_files]
        else:
            self.images = [img for img, labels in test_files]
            self.labels = [labels for img, labels in test_files]


    def __getitem__(self, idx):
        img_path = self.images[idx]
        img = cv2.imread(img_path)

        if self.transform:
            img = self.transform(img)

        labels = self.labels[idx]
        return img, *labels

    def __len__(self):
        return len(self.images)

In [7]:
batch_size =32
transform = transforms.Compose([
    transforms.ToTensor(),
     transforms.Resize(224),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

train_dataset = facedataset(train_data, train= True, transform= transform)
validation_dataset =facedataset(train_data, train= False, transform= transform)
test_dataset  = facedataset(validation_data, test= True,transform= transform)

train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_data_loader = DataLoader(validation_dataset,batch_size=batch_size,)
test_data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [8]:
print(len(train_data_loader))
print(len(validation_data_loader))
print(len(test_data_loader))

2131
533
417


In [9]:
label_classes = []
data_ex = next(iter(validation_dataset))
for i in range(1,len(data_ex)):
    labelSize = data_ex[i].size()
    label_classes.append(labelSize.numel())
print(label_classes)

[8, 27, 2, 4, 3, 8, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


C:\Users\Rui\.conda\envs\hufsG\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [11]:
!pip install torchsummary

In [10]:
class MultiOutputModel(nn.Module):
    def __init__(self, label_classes):
        super(MultiOutputModel, self).__init__()
        
        vgg16_model = models.vgg16(pretrained=True)
        for param in vgg16_model.parameters():
            param.requires_grad = False;
            
        self.features = vgg16_model.features

        self.flatten = nn.Flatten()
            
        classifiers = []
        for label_class in label_classes:
            classifiers.append(nn.Sequential(
                nn.Linear(25088, label_class),
            ))

        self.classifiers = nn.ModuleList(classifiers)

    def forward(self, x):
        x = self.features(x)
        x = self.flatten(x)
        
        outputs = [classifier(x) for classifier in self.classifiers]

        return tuple(outputs)

In [43]:
from torchsummary import summary

label_classes = [8, 27, 2, 4, 3, 8, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
'''
data_ex = next(iter(validation_dataset))
for i in range(1,len(data_ex)):
    labelSize = data_ex[i].size()
    label_classes.append(labelSize.numel())
print(label_classes)
'''
model = MultiOutputModel(label_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("Using device:", device)

summary(model, (3, 224, 224))
print(model)

Using device: cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14

In [32]:
num_epochs = 20
params = [p for p in model.parameters() if p.requires_grad]
print(params)
optimizer = torch.optim.SGD(params, lr=0.001,
                                momentum=0.9, weight_decay=0.0005)
criterion = nn.CrossEntropyLoss()
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)

[Parameter containing:
tensor([[ 0.2182,  0.0739, -0.1407,  ..., -0.2717, -0.4562,  0.0300],
        [-0.0149, -0.0256,  0.0256,  ..., -0.0156, -0.0337,  0.0693],
        [ 0.0961,  0.2174,  0.0590,  ...,  0.0709,  0.0701,  0.1666],
        ...,
        [ 0.0364,  0.1109,  0.1155,  ...,  0.1096,  0.2287,  0.0670],
        [-0.1334, -0.1567, -0.0604,  ..., -0.0971,  0.1199, -0.0038],
        [ 0.0959, -0.1972,  0.1799,  ..., -0.0983,  0.1354, -0.3087]],
       device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.0442, -0.0065, -0.0076,  0.0491, -0.0346,  0.0249, -0.1128,  0.1300],
       device='cuda:0', requires_grad=True), Parameter containing:
tensor([[-0.0519,  0.0733,  0.0460,  ..., -0.1055,  0.0325,  0.0552],
        [-0.0064, -0.0206, -0.0599,  ...,  0.0057, -0.2704, -0.0634],
        [ 0.0928, -0.0241, -0.0811,  ..., -0.2213, -0.1899,  0.0057],
        ...,
        [ 0.0316,  0.1695,  0.1535,  ..., -0.0691,  0.0890,  0.0116],
        [-0.1191, -0.0340, -0.1168,

In [ ]:
#다중 출력 모델
epoch_list = []
loss_list = []

train_loss_list = []
train_accuracy_list = []

validation_loss_list = []
validation_accuracy_list = []

'''
plt.xlabel('Epoch')
plt.ylabel('Total Loss')
plt.title('Training Loss Over Epochs')
'''


for epoch in range(num_epochs):
    # Training

        
    model.train()
    total_loss = 0.0
    for images, *labels in train_data_loader:  # Use *labels to unpack the list of label tensors
        images = images.to(device)
        labels = [label.to(device) for label in labels]  # Move each label tensor to the device
        # Forward pass
        outputs = model(images)

        integer_labels = [torch.argmax(label, dim=1) for label in labels]
        
        # Calculate loss for each output
        losses = [criterion(output, label) for output, label in zip(outputs, integer_labels)]
        
        # Sum the losses
        total_loss = sum(losses)
        '''

        loss_list.append(total_loss.item()/(len(loss_list)+1))
        if len(epoch_list) == 0:
            epoch_list.append(1/len(train_data_loader))
        else:
            epoch_list.append(epoch_list[-1] + 1/len(train_data_loader))
            
        plt.figure(figsize=(15, 5))
        plt.plot(epoch_list,loss_list, label='Training Loss', color='blue')
        plt.legend(loc='upper right')
        plt.xticks([i for i in range(num_epochs+1)])
        plt.yticks([i for i in range(3)])
        plt.ylim(0,3)
        plt.show()
        clear_output(wait=True)
        '''

        # Backward pass and optimization
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
        scheduler.step()
        
        
        corrects = [torch.sum(torch.argmax(output, 1) == torch.argmax(label, 1)).item() for output, label in zip(outputs, labels)]
        total_corrects = [total + correct for total, correct in zip(total_corrects, corrects)]

    avg_train_loss = total_loss / len(train_data_loader.dataset)
    train_accuracies = [total_correct / len(train_data_loader.dataset) for total_correct in total_corrects]

    train_loss_list.append(avg_train_loss)
    train_accuracy_list.append(train_accuracies)
    
    # Validation
    model.eval()
    total_corrects = [0] * len(labels)
    total_loss = 0.0

    with torch.no_grad():
        for images, *labels in validation_data_loader:
            images = images.to(device)
            labels = [label.to(device) for label in labels]

            # Forward pass
            outputs = model(images)

            integer_labels = [torch.argmax(label, dim=1) for label in labels]

            # Calculate loss for each output
            losses = [criterion(output, label) for output, label in zip(outputs, integer_labels)]

            # Sum the losses
            total_loss += sum(losses).item()

            # Calculate total correct predictions for each output
            corrects = [torch.sum(torch.argmax(output, 1) == torch.argmax(label, 1)).item() for output, label in zip(outputs, labels)]
            total_corrects = [total + correct for total, correct in zip(total_corrects, corrects)]

    # Calculate average loss and accuracy for each output
    avg_validation_loss = total_loss / len(validation_data_loader.dataset)
    validation_accuracies = [total_correct / len(validation_data_loader.dataset) for total_correct in total_corrects]

    validation_loss_list.append(avg_validation_loss)
    validation_accuracy_list.append(validation_accuracies)

    # Print results
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_validation_loss:.4f}")
    print("Training Accuracy : ")
    for i, accuracy in enumerate(train_accuracies):
        print(f"Accuracy{i + 1}: {accuracy * 100:.2f}%, ", end="")
    print()
    print("Validation Accuracy : ")
    for i, accuracy in enumerate(validation_accuracies):
        print(f"Accuracy{i + 1}: {accuracy * 100:.2f}%, ", end="")
    print()

Epoch [1/20], Training Loss: 0.0004, Validation Loss: 0.7448
Training Accuracy : 
Accuracy1: 96.12%, Accuracy2: 109.66%, Accuracy3: 122.28%, Accuracy4: 89.62%, Accuracy5: 108.28%, Accuracy6: 90.81%, Accuracy7: 107.71%, Accuracy8: 115.81%, Accuracy9: 103.00%, Accuracy10: 110.61%, Accuracy11: 122.17%, Accuracy12: 111.45%, Accuracy13: 107.97%, Accuracy14: 112.62%, Accuracy15: 113.71%, Accuracy16: 119.43%, Accuracy17: 109.06%, Accuracy18: 115.76%, Accuracy19: 121.53%, Accuracy20: 118.93%, Accuracy21: 112.92%, Accuracy22: 113.96%, Accuracy23: 119.26%, Accuracy24: 119.10%, Accuracy25: 126.60%, Accuracy26: 126.54%, 
Validation Accuracy : 
Accuracy1: 73.07%, Accuracy2: 83.84%, Accuracy3: 93.20%, Accuracy4: 68.02%, Accuracy5: 83.87%, Accuracy6: 69.54%, Accuracy7: 81.92%, Accuracy8: 88.39%, Accuracy9: 79.06%, Accuracy10: 84.52%, Accuracy11: 93.69%, Accuracy12: 85.39%, Accuracy13: 82.35%, Accuracy14: 86.26%, Accuracy15: 86.63%, Accuracy16: 91.45%, Accuracy17: 83.02%, Accuracy18: 88.39%, Accuracy1

In [56]:
torch.save(model.state_dict(), 'multi_output_model_2.pt')
torch.save(model.state_dict(), 'multi_output_model_2.pth')

In [31]:
# 저장한 모델 불러오기

from torchsummary import summary
model = MultiOutputModel(label_classes)

model.load_state_dict(torch.load('multi_output_model_2.pth'))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("Using device:", device)

summary(model, (3, 224, 224))
print(model)

Using device: cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14

In [11]:
torch.save(model.state_dict(), 'multi_output_model.pt')